# 🕸️ GraphRAG Toolkit Demo

This notebook demonstrates how to use Microsoft's **GraphRAG** (Graph-based Retrieval-Augmented Generation) toolkit. GraphRAG enhances traditional RAG by building a knowledge graph from your documents, enabling more nuanced and comprehensive answers to complex queries.

## What is GraphRAG?

GraphRAG is an advanced RAG approach that:
- **Extracts entities and relationships** from documents to build a knowledge graph
- **Creates community summaries** for hierarchical understanding
- **Supports two query modes**:
  - **Local Search**: Best for specific questions about particular entities
  - **Global Search**: Best for broad questions requiring synthesis across the corpus

---


## 📦 1. Installation

First, let's install the GraphRAG toolkit and its dependencies.


In [ ]:
# Install GraphRAG toolkit
!pip install graphrag --quiet

# Additional dependencies
!pip install python-dotenv pyyaml --quiet


## 🔧 2. Environment Setup

GraphRAG requires an LLM (Language Model) and an embedding model. We'll configure these using environment variables.

**Supported LLM providers:**
- OpenAI
- Azure OpenAI
- Other OpenAI-compatible APIs


In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
load_dotenv()

# Set your OpenAI API key (uncomment and set your key)
# os.environ["GRAPHRAG_API_KEY"] = "your-openai-api-key-here"

# Alternatively, you can set it directly:
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# Verify the API key is set
api_key = os.environ.get("GRAPHRAG_API_KEY") or os.environ.get("OPENAI_API_KEY")
if api_key:
    print("✅ API key is configured")
else:
    print("⚠️  Warning: No API key found. Please set GRAPHRAG_API_KEY or OPENAI_API_KEY")


## 📁 3. Project Structure Setup

GraphRAG expects a specific project structure. Let's create it.


In [ ]:
# Define project directories
PROJECT_DIR = Path.cwd()
INPUT_DIR = PROJECT_DIR / "input"
OUTPUT_DIR = PROJECT_DIR / "output"

# Create directories
INPUT_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"📂 Project directory: {PROJECT_DIR}")
print(f"📂 Input directory: {INPUT_DIR}")
print(f"📂 Output directory: {OUTPUT_DIR}")


## 📄 4. Sample Data Preparation

For this demo, we'll create a sample text document. You can replace this with your own documents.


In [ ]:
# Sample text about a fictional tech company for demonstration
sample_text = """
# TechCorp Innovation Report 2025

## Company Overview

TechCorp is a leading technology company founded in 2015 by Sarah Chen and Michael Rodriguez in San Francisco. 
The company specializes in artificial intelligence solutions for enterprise customers. With over 5,000 employees 
across 20 offices worldwide, TechCorp has become a major player in the AI industry.

## Leadership Team

Sarah Chen serves as the CEO and has led the company through multiple successful funding rounds. She previously 
worked at Google and Stanford AI Lab. Michael Rodriguez, the CTO, oversees all technical operations and R&D. 
He holds a PhD in Machine Learning from MIT.

The CFO, Jennifer Park, joined in 2019 from Goldman Sachs. She has been instrumental in the company's financial 
growth and successful IPO in 2023. David Thompson leads the Sales division and has expanded the customer base 
to include Fortune 500 companies like Amazon, Microsoft, and Walmart.

## Products and Services

TechCorp's flagship product, "AIAssist Pro", is an enterprise AI assistant that helps companies automate 
customer service operations. It uses advanced natural language processing and has been deployed by over 
200 enterprise customers.

"DataSense Analytics" is the company's second major product, offering predictive analytics for supply chain 
optimization. Major clients include Walmart and Target, who have reported 30% efficiency improvements.

The newest product, "SecureAI", launched in 2024, focuses on AI-powered cybersecurity. It has already 
attracted partnerships with three major banks: JPMorgan Chase, Bank of America, and Wells Fargo.

## Research and Development

TechCorp's R&D division, led by Dr. Emily Watson, has published over 50 papers in top AI conferences. 
The team recently made a breakthrough in efficient transformer architectures, reducing compute costs by 40%.

The company collaborates with Stanford University, MIT, and Carnegie Mellon on various research projects. 
Dr. Watson's team includes researchers from DeepMind, OpenAI, and Google Brain.

## Financial Performance

In 2024, TechCorp reported revenue of $2.5 billion, a 45% increase from the previous year. The company's 
market cap reached $50 billion after the successful IPO. Major investors include Sequoia Capital, 
Andreessen Horowitz, and SoftBank Vision Fund.

## Future Plans

TechCorp plans to expand into the healthcare AI market in 2025, with partnerships already in place with 
Mayo Clinic and Cleveland Clinic. The company is also developing autonomous systems for logistics, 
working with FedEx and UPS on pilot programs.

Sarah Chen announced plans to open new R&D centers in London, Singapore, and Tel Aviv to attract 
global talent and serve international customers better.
"""

# Save the sample text to the input directory
input_file = INPUT_DIR / "techcorp_report.txt"
input_file.write_text(sample_text)

print(f"✅ Sample document saved to: {input_file}")
print(f"📝 Document length: {len(sample_text)} characters")


## ⚙️ 5. Configuration

GraphRAG uses a `settings.yaml` file for configuration. Let's create one with sensible defaults.


In [ ]:
import yaml

# GraphRAG configuration
settings = {
    "llm": {
        "api_key": "${GRAPHRAG_API_KEY}",  # Uses environment variable
        "type": "openai_chat",
        "model": "gpt-4o-mini",  # Cost-effective model for demo
        "model_supports_json": True,
        "max_tokens": 4000,
        "temperature": 0,
    },
    "embeddings": {
        "llm": {
            "api_key": "${GRAPHRAG_API_KEY}",
            "type": "openai_embedding",
            "model": "text-embedding-3-small",
        }
    },
    "input": {
        "type": "file",
        "file_type": "text",
        "base_dir": "input",
        "file_encoding": "utf-8",
        "file_pattern": ".*\\.txt$"
    },
    "storage": {
        "type": "file",
        "base_dir": "output"
    },
    "cache": {
        "type": "file",
        "base_dir": "cache"
    },
    "reporting": {
        "type": "file",
        "base_dir": "logs"
    },
    "chunks": {
        "size": 1200,
        "overlap": 100
    },
    "entity_extraction": {
        "max_gleanings": 1
    },
    "claim_extraction": {
        "enabled": True
    },
    "community_reports": {
        "max_length": 2000
    },
    "cluster_graph": {
        "max_cluster_size": 10
    }
}

# Save settings to YAML file
settings_file = PROJECT_DIR / "settings.yaml"
with open(settings_file, 'w') as f:
    yaml.dump(settings, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved to: {settings_file}")
print("\n📋 Settings preview:")
print(yaml.dump(settings, default_flow_style=False, sort_keys=False))


## 🔍 6. Initialize GraphRAG

Alternatively, you can use the CLI to initialize a project with `graphrag init --root .`

Let's verify the GraphRAG library is installed correctly.


In [ ]:
# Check if we can import graphrag
try:
    import graphrag
    print(f"✅ GraphRAG version: {graphrag.__version__ if hasattr(graphrag, '__version__') else 'installed'}")
except ImportError as e:
    print(f"❌ GraphRAG not installed: {e}")
    print("Run: pip install graphrag")


## 🏗️ 7. Indexing Documents

The indexing process:
1. **Chunks** documents into smaller pieces
2. **Extracts** entities and relationships using the LLM
3. **Builds** a knowledge graph
4. **Creates** community summaries using the Leiden algorithm
5. **Generates** embeddings for semantic search

⚠️ **Note**: Indexing uses LLM API calls and may take several minutes and incur costs.


In [ ]:
# Method 1: Using CLI (recommended for larger documents)
# Run this in a terminal: graphrag index --root .

# Method 2: Using Python API
import subprocess
import sys

def run_graphrag_index():
    """Run GraphRAG indexing process."""
    print("🚀 Starting GraphRAG indexing...")
    print("This may take several minutes depending on document size.\n")
    
    try:
        result = subprocess.run(
            [sys.executable, "-m", "graphrag", "index", "--root", str(PROJECT_DIR)],
            capture_output=True,
            text=True,
            cwd=str(PROJECT_DIR)
        )
        
        if result.returncode == 0:
            print("✅ Indexing completed successfully!")
            print(result.stdout)
        else:
            print("❌ Indexing failed:")
            print(result.stderr)
            
    except Exception as e:
        print(f"❌ Error running indexing: {e}")

# Uncomment the line below to run indexing
# run_graphrag_index()


### Alternative: Run indexing via command line

You can also run the indexing from the command line:


In [ ]:
# Uncomment to run indexing via shell
# !graphrag index --root .


## 🔎 8. Querying the Knowledge Graph

GraphRAG supports two types of queries:

### Local Search
- Best for **specific questions** about particular entities or facts
- Uses entity embeddings and local graph context
- Example: "Who is the CEO of TechCorp?"

### Global Search  
- Best for **broad questions** requiring synthesis across the entire corpus
- Uses community summaries for high-level understanding
- Example: "What are the main business areas of TechCorp?"


In [ ]:
def run_graphrag_query(query: str, method: str = "local"):
    """
    Run a GraphRAG query.
    
    Args:
        query: The question to ask
        method: 'local' or 'global' search method
    """
    print(f"🔍 Query: {query}")
    print(f"📊 Method: {method.upper()} search\n")
    
    try:
        result = subprocess.run(
            [
                sys.executable, "-m", "graphrag", "query",
                "--root", str(PROJECT_DIR),
                "--method", method,
                "--query", query
            ],
            capture_output=True,
            text=True,
            cwd=str(PROJECT_DIR)
        )
        
        if result.returncode == 0:
            print("📝 Response:")
            print("-" * 50)
            print(result.stdout)
        else:
            print("❌ Query failed:")
            print(result.stderr)
            
    except Exception as e:
        print(f"❌ Error running query: {e}")


### Example Queries

Try these example queries after indexing is complete:


In [ ]:
# Local Search - specific entity questions
# Uncomment after indexing is complete

# run_graphrag_query("Who is Sarah Chen and what is her role at TechCorp?", method="local")


In [ ]:
# run_graphrag_query("What products does TechCorp offer?", method="local")


In [ ]:
# Global Search - broad synthesis questions
# run_graphrag_query("What are the main themes and topics discussed in the document?", method="global")


In [ ]:
# run_graphrag_query("Summarize TechCorp's business strategy and future plans.", method="global")


## 📊 9. Exploring the Knowledge Graph

After indexing, GraphRAG creates several output files that you can explore:


In [ ]:
import pandas as pd

def explore_graphrag_outputs():
    """Explore the GraphRAG output files."""
    output_artifacts = OUTPUT_DIR / "artifacts"
    
    if not output_artifacts.exists():
        print("⚠️  Output directory not found. Run indexing first.")
        return
    
    # List available output files
    print("📁 Available output files:")
    for file in output_artifacts.glob("*.parquet"):
        print(f"  - {file.name}")
    
    # Try to load and display entities
    entities_file = output_artifacts / "create_final_entities.parquet"
    if entities_file.exists():
        print("\n🏷️  Extracted Entities:")
        entities_df = pd.read_parquet(entities_file)
        print(entities_df[['name', 'type', 'description']].head(10))
    
    # Try to load and display relationships
    rels_file = output_artifacts / "create_final_relationships.parquet"
    if rels_file.exists():
        print("\n🔗 Extracted Relationships:")
        rels_df = pd.read_parquet(rels_file)
        print(rels_df[['source', 'target', 'description']].head(10))

# Uncomment after indexing
# explore_graphrag_outputs()


## 🎨 10. Visualizing the Knowledge Graph

Let's create a visualization of the extracted knowledge graph.


In [ ]:
# Install visualization libraries
!pip install networkx matplotlib --quiet


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def visualize_knowledge_graph():
    """Create a visualization of the knowledge graph."""
    output_artifacts = OUTPUT_DIR / "artifacts"
    
    entities_file = output_artifacts / "create_final_entities.parquet"
    rels_file = output_artifacts / "create_final_relationships.parquet"
    
    if not entities_file.exists() or not rels_file.exists():
        print("⚠️  Output files not found. Run indexing first.")
        return
    
    # Load data
    entities_df = pd.read_parquet(entities_file)
    rels_df = pd.read_parquet(rels_file)
    
    # Create graph
    G = nx.Graph()
    
    # Add nodes
    for _, row in entities_df.iterrows():
        G.add_node(row['name'], type=row.get('type', 'UNKNOWN'))
    
    # Add edges
    for _, row in rels_df.iterrows():
        G.add_edge(row['source'], row['target'])
    
    # Create visualization
    plt.figure(figsize=(15, 10))
    
    # Color nodes by type
    node_types = nx.get_node_attributes(G, 'type')
    unique_types = list(set(node_types.values()))
    color_map = plt.cm.get_cmap('tab10')
    colors = [color_map(unique_types.index(node_types.get(node, 'UNKNOWN')) % 10) for node in G.nodes()]
    
    # Layout
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Draw
    nx.draw(G, pos, 
            node_color=colors,
            node_size=1000,
            font_size=8,
            font_weight='bold',
            with_labels=True,
            edge_color='gray',
            alpha=0.7)
    
    plt.title("GraphRAG Knowledge Graph", fontsize=16, fontweight='bold')
    
    # Add legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                   markerfacecolor=color_map(i % 10), 
                                   markersize=10, label=t) 
                       for i, t in enumerate(unique_types)]
    plt.legend(handles=legend_elements, loc='upper left', title='Entity Types')
    
    plt.tight_layout()
    plt.savefig(PROJECT_DIR / 'knowledge_graph.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Graph Statistics:")
    print(f"   Nodes: {G.number_of_nodes()}")
    print(f"   Edges: {G.number_of_edges()}")
    print(f"   Entity Types: {unique_types}")

# Uncomment after indexing
# visualize_knowledge_graph()


## 11. Cleanup

Optional: Remove generated files to start fresh.


In [ ]:
import shutil

def cleanup():
    """Remove all generated files."""
    dirs_to_remove = ['output', 'cache', 'logs']
    
    for dir_name in dirs_to_remove:
        dir_path = PROJECT_DIR / dir_name
        if dir_path.exists():
            shutil.rmtree(dir_path)
            print(f"Removed: {dir_path}")
    
    print("Cleanup complete!")

# Uncomment to run cleanup
# cleanup()


---

## Summary

In this notebook, we covered:

1. **Installation** of GraphRAG toolkit
2. **Configuration** of LLM and embedding models
3. **Project setup** with proper directory structure
4. **Document indexing** to build the knowledge graph
5. **Querying** using local and global search methods
6. **Exploration** of extracted entities and relationships
7. **Visualization** of the knowledge graph

## Resources

- [GraphRAG Documentation](https://microsoft.github.io/graphrag/)
- [GraphRAG GitHub Repository](https://github.com/microsoft/graphrag)
- [GraphRAG Research Paper](https://arxiv.org/abs/2404.16130)

## Tips

- Use **Local Search** for specific entity lookups
- Use **Global Search** for summarization and thematic questions
- Adjust `chunk_size` based on your document structure
- Consider using `gpt-4o` for better extraction quality
- Monitor API costs during indexing of large document sets
